ПРИМЕР РАБОТЫ С KAFKA С ПОМОЩЬЮ PYTHON

Данильченко Вадим

In [1]:
!pip install kafka-python

In [1]:
import logging
from kafka import KafkaProducer
from kafka import KafkaConsumer

import requests
from bs4 import BeautifulSoup
import html
from time import sleep

import re, json

In [3]:
# добавим логирование
logging.basicConfig(level=logging.INFO)

In [10]:
# создадим producer и запишем несколько сообщений в топик test_topic

producer = KafkaProducer(bootstrap_servers=['0.0.0.0:9092'], api_version=(1,0,0))
producer.send('test_topic', b'Hello, World!').get(timeout=10)
producer.send('test_topic', key=b'msg2', value=b'one more test message')

INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: Closing connection. 
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=0.0.0.0:9092 <connecting> [IPv4 ('0.0.0.0', 9092)]>: connecting to 0.0.0.0:9092 [('0.0.0.0', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=0.0.0.0:9092 <connecting> [IPv4 ('0.0.0.0', 9092)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=0.0.0.0:9092 <connected> [IPv4 ('0.0.0.0', 9092)]>: Closing connection. 


In [12]:
# запустим consumer, который будет слушать топик test_topic и выводить value из полученного сообщения
from kafka import KafkaConsumer
consumer = KafkaConsumer('test_topic',
                        bootstrap_servers=['0.0.0.0:9092'],
                        group_id=None,
                        auto_offset_reset='earliest',
                        enable_auto_commit=False)

for message in consumer:
    print(message.value)

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=0.0.0.0:9092 <connecting> [IPv4 ('0.0.0.0', 9092)]>: connecting to 0.0.0.0:9092 [('0.0.0.0', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=0.0.0.0:9092 <connecting> [IPv4 ('0.0.0.0', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('test_topic',)
INFO:kafka.consumer.subscription_state:Updated partition assignment: [TopicPartition(topic='test_topic', partition=0)]
INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.

b'Hello, World!'
b'one more test message'


KeyboardInterrupt: 

--------------------------------------------------------------------------------------------------------------------

будем парсить сайт с рецептами, забирать рецепты салатов, складывать html в топик кафки, преобразовывать в джейсон и передавать в другой топик (json_recipes), следующий consumer будет забирать информацию о калорийности, список ингредиентов и время готовки, также он будет выдавать alert, если количество калорий выше порога

In [2]:
# функция забирает данные со страницы сайта по полученной ссылке

def fetch_raw(recipe_url):
    html = None
    print('Processing..{}'.format(recipe_url))
    try:
        r = requests.get(recipe_url, headers=headers)
        if r.status_code == 200:
            html = r.text
    except Exception as ex:
        print('Exception while accessing raw html')
        print(str(ex))
    finally:
        return html.strip()

In [22]:
# функция парсит сайт, находит нужный объект и забирает ссылки на страницы рецептов

def get_recipes():
    recipies = []
    links = []
    url = 'https://www.allrecipes.com/recipes/96/salad/'
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'lxml')
            links = soup.select('.category-page-item-content a')
            idx = 0
            for link in links:
                sleep(2)
                tmp_link = link['href']
                if tmp_link not in links:
                    links.append(tmp_link)
                    recipe = fetch_raw(tmp_link)
                    recipies.append(recipe)
                    idx += 1
                else:
                    continue
                    
                if idx > 2:
                    break
    except Exception as ex:
        print('Exception in get_recipes')
        print(str(ex))
    finally:
        return recipies

In [23]:
# функция публикует сообщение в выбранный топик

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

In [24]:
# функция создает producer

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['0.0.0.0:9092'],api_version=(1, 0, 0))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [25]:
# собираем все кусочки воедино - парсим сайт, забираем данные рецептов салатов, публикуем в топик raw_recipes

headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.182 Safari/537.36',
        'Pragma': 'no-cache'
    }

all_recipes = get_recipes()
if len(all_recipes) > 0:
    kafka_producer = connect_kafka_producer()
    for recipe in all_recipes:
        publish_message(kafka_producer, 'raw_recipes', 'raw', recipe.strip())
    if kafka_producer is not None:
        kafka_producer.close()

Accessing list
Processing..https://www.allrecipes.com/recipe/19607/thai-beef-salad/
Processing..https://www.allrecipes.com/recipe/14373/greek-salad-i/
Processing..https://www.allrecipes.com/recipe/150738/wedge-salad-with-elegant-blue-cheese-dressing/
Message published successfully.
Message published successfully.
Message published successfully.


In [99]:
# первый consumer, который полуучает данные html и переобразовывает их в json, 
# который передает их дальше в другой топик json_recipes
consumer = KafkaConsumer('raw_recipes',
                         bootstrap_servers=['0.0.0.0:9092'],
                         group_id=None,
                         auto_offset_reset='earliest',
                         enable_auto_commit=False
                        )

def preprocess_html(html_data):
    soup = BeautifulSoup(html_data, 'lxml')
    
    # извлечем ингредиенты
    ingredients = []
    for text in soup.select('.ingredients-item-name'):
        ingredients.append(text.get_text().strip())

    # извлечем время приготовления
    coocking_section = soup.select('.recipe-meta-item')
    for elem in coocking_section:
        if elem.select('.recipe-meta-item-header')[0].text=='total:':
            coocking_time = elem.select('.recipe-meta-item-body')[0].text.strip()
            break
            
    # извлечем калории
    for elem in soup.select('.section-body'):
        if 'calories' in elem.get_text():
            calories = [i.strip() for i in elem.get_text().split(';') if 'calories' in i][0]
    
    # найдем название салата
    salad_name = soup.find('h1').get_text().strip()
    
    return '; '.join(ingredients), coocking_time, calories, salad_name
    

# создадим подключение
kafka_producer = connect_kafka_producer()

# передадим результат в формате json в следующий топик json_recipes
for message in consumer:
    ingredients, coocking_time, calories, salad_name = preprocess_html(message.value)
    
    print(message.key,'\n', ingredients,'\n', coocking_time,'\n', calories,'\n', salad_name,'\n')
    json_msg = json.dumps({'ingredients': ingredients,
                           'coocking_time': coocking_time,
                           'calories': calories
                          })
    
    
    publish_message(kafka_producer, 'json_recipes', salad_name, json_msg)

# закроем подключение
if kafka_producer is not None:
    kafka_producer.close()

b'raw' 
 2 green onions, chopped; 1 lemon grass, cut into 1 inch pieces; 1 cup chopped fresh cilantro; 1 cup chopped fresh mint leaves; 1 cup lime juice; ⅓ cup fish sauce; 1 tablespoon sweet chili sauce; ½ cup white sugar; 1 ½ pounds (1 inch thick) steak fillet; 1 head leaf lettuce - rinsed, dried and torn into bite-size pieces; ½ English cucumber, diced; 1 pint cherry tomatoes 
 3 hrs 45 mins 
 211 calories 
 Thai Beef Salad 

Message published successfully.
b'raw' 
 1 head romaine lettuce- rinsed, dried and chopped; 1 red onion, thinly sliced; 1 (6 ounce) can pitted black olives; 1 green bell pepper, chopped; 1 red bell pepper, chopped; 2 large tomatoes, chopped; 1 cucumber, sliced; 1 cup crumbled feta cheese; 6 tablespoons olive oil; 1 teaspoon dried oregano; 1 lemon, juiced; ground black pepper to taste 
 20 mins 
 265 calories 
 Greek Salad I 

Message published successfully.
b'raw' 
 ½ pound crumbled blue cheese; ¼ cup sour cream; ⅓ cup buttermilk; ½ cup mayonnaise; ¼ cup red win

KeyboardInterrupt: 

In [111]:
# создадим consumer, который будет принимать сообщения из топика json_recipes в формате json, 
# после чего будет сигнализировать, если количество калорий больше threshold(300)
consumer_json = KafkaConsumer('json_recipes',
                         bootstrap_servers=['0.0.0.0:9092'],
                         group_id=None,
                         auto_offset_reset='earliest',
                         enable_auto_commit=False
                        )


threshold = 300
for message in consumer_json:
    msg = json.loads(message.value)
    print(f"salad name: {message.key.decode('utf-8')}")
    # 
    if int(re.sub("[^0-9]", "", msg['calories']))>threshold:
        print("{:!^100}".format(' FAT ALERT '))
    print("recieved message from topic json_recipes: \n", str(msg), "\n")

salad name: Thai Beef Salad
recieved message from topic json_recipes: 
 {'ingredients': '2 green onions, chopped; 1 lemon grass, cut into 1 inch pieces; 1 cup chopped fresh cilantro; 1 cup chopped fresh mint leaves; 1 cup lime juice; ⅓ cup fish sauce; 1 tablespoon sweet chili sauce; ½ cup white sugar; 1\u2009½ pounds (1 inch thick) steak fillet; 1 head leaf lettuce - rinsed, dried and torn into bite-size pieces; ½ English cucumber, diced; 1 pint cherry tomatoes', 'coocking_time': '3 hrs 45 mins', 'calories': '211 calories'} 

salad name: Greek Salad I
recieved message from topic json_recipes: 
 {'ingredients': '1 head romaine lettuce- rinsed, dried and chopped; 1 red onion, thinly sliced; 1 (6 ounce) can pitted black olives; 1 green bell pepper, chopped; 1 red bell pepper, chopped; 2 large tomatoes, chopped; 1 cucumber, sliced; 1 cup crumbled feta cheese; 6 tablespoons olive oil; 1 teaspoon dried oregano; 1 lemon, juiced; ground black pepper to taste', 'coocking_time': '20 mins', 'calo

KeyboardInterrupt: 